In [1]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
from dask.distributed import Client, progress, wait
from IPython.display import display, HTML
from sklearn.metrics import roc_auc_score
import project_path
from src.util.generate_connected_graph import generate_connected_graph
from src.util.generate_lr_data import generate_low_rank_data
from src.util.generate_anomaly import generate_spatio_temporal_anomaly
from src.algos.lr_stss import lr_stss

In [2]:
client = Client(threads_per_worker=1, n_workers=14)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 14
Total threads: 14,Total memory: 15.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55289,Workers: 14
Dashboard: http://127.0.0.1:8787/status,Total threads: 14
Started: Just now,Total memory: 15.73 GiB
Comm: tcp://127.0.0.1:55370,Total threads: 1
Dashboard: http://127.0.0.1:55387/status,Memory: 1.12 GiB
Nanny: tcp://127.0.0.1:55292,


In [13]:
## Control Variables
gt = 'grid'
graph_param = (8,5)
dtn = 4
rad =1
amp =1
window = 'boxcar'
distribution = 'bernoulli'
local_distribution = 'constant'
dims = (40,24,7,4)
ranks = (8,8,5,4)
NoA = 100
NoG = 1
NoT = 5
time_m = 2
local_m = 1
lda_2 = 100
psi = 20
maxit = 300
maxit2 = 40
## Independent variables
### Hyperparameters
lda_1 = np.logspace(1,-4,3)
lda_l = np.logspace(1,-4,4)
lda_t = np.logspace(1,-4,4)

metadata = {'graph_type': gt,
            'graph_param': graph_param,
            'number_of_anomalies': NoA,
            'anomaly_duration': dtn,
            'anomaly_radius': rad,
            'anomaly_amplitude': amp,
            'window': window,
            'distribution': distribution,
            'local_distribution': local_distribution,
            'number_of_graphs': NoG,
            'number_of_trials': NoT,
            'ranks':ranks,
            'dims':dims,
            'time_mode': time_m,
            'local_mode': local_m,
            'lda_2':lda_2,
            'lda_1':lda_1,
            'lda_l':lda_l,
            'lda_t':lda_t,
            'psi': psi,
            'maxit':maxit,
            'maxit2':maxit2,
            }

# Define experiment function
def run_exp(X, Y, an_m, inputs):
    G = inputs['G']
    A = nx.adjacency_matrix(G)
    Deg = np.diag(np.asarray(np.sum(A,axis=1)).ravel())
    Dsq = np.linalg.inv(np.sqrt(Deg))
    An = Dsq@A@Dsq
    Y = inputs['Y']
    Y = np.ma.masked_array(Y, mask=np.zeros(dims,dtype=bool))
    X = inputs['X']
    an_m = inputs['an_m']
    psi = inputs['psi']
    res = lr_stss(Y, An, time_m,local_m, verbose=0, max_it2=40, max_it=300,
        lda2=inputs['lda_2'], lda1=inputs['lda_1'], lda_t=inputs['lda_t'],
        lda_loc=inputs['lda_l'], psis=[psi,psi,psi,psi])
    
    result = {'graph_type': gt,
              'auc': roc_auc_score(an_m.ravel(),np.abs(res['S']).ravel()),
              'rec_err': np.linalg.norm(res['X']-X)/np.linalg.norm(X),
              'anomaly_cardinality': an_m.sum(),
              'number_of_anomalies': NoA,
              'anomaly_duration': dtn,
              'anomaly_radius': rad,
              'graph_seed': inputs['graph_seed'],
              'anomaly_seed': inputs['anomaly_seed'],
              'anomaly_amplitude': amp,
              'lda_1': inputs['lda_1'],
              'lda_2': inputs['lda_2'],
              'lda_l': inputs['lda_l'],
              'lda_t': inputs['lda_t'],
              'psi_1': inputs['psi'], 'psi_2': inputs['psi'],
              'psi_3': inputs['psi'], 'psi_4': inputs['psi'],
              'maxit': maxit, 'maxit2': maxit2,
              'it': res['it']}
    return result

In [15]:
try:
    del futures
finally:
    pass
futures = []
seed = 984854251
input = {}
for i in range(NoG):
    G,sd = generate_connected_graph(graph_param, gt, radius=graph_param, seed=seed)
    input['G'] = G
    input['graph_type'] = gt
    input['graph_seed'] = sd
    seed = sd
    for j in range(NoT):
        # Generate low rank normal data
        X = generate_low_rank_data(dims, ranks, seed)
        X = 1*X/np.std(X)
        input['X'] = X
        # Generate anomaly
        an, an_m = generate_spatio_temporal_anomaly(dims, G, NoA, duration=dtn,
                                                    radius=rad, seed=seed,
            time_m=time_m, local_m=local_m, window_type='boxcar', amplitude=amp,
            distribution=distribution, local_dist='constant')
        input['anomaly_seed'] = seed
        input['an_m'] = an_m
        Y = X+an
        # Y = np.ma.masked_array(Y, mask=np.zeros(dims,dtype=bool))
        input['Y'] = Y
        seed +=1
        for k1 in range(len(lda_1)):
            for k2 in range(len(lda_l)):
                for k3 in range(len(lda_t)):
                    input['lda_1'] = lda_1[k1]
                    input['lda_l'] = lda_l[k2]
                    input['lda_t'] = lda_t[k3]
                    input['lda_2'] = lda_2
                    input['psi'] = psi
                    futures.append(client.submit(run_exp, X, Y, an_m, input))
            

Graph is connected.


c:\Users\merti\ResearchRepos\TensorAnomalyDetection\src\util\generate_anomaly.py:165: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


In [16]:
progress(futures)

VBox()

In [8]:
fname = 'lr_stss_hyperparameter_study.hdf5'
key = 'grid'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)

for i in range(len(futures)):
    df = pd.DataFrame(futures[i].result(), index=[i])
    with pd.HDFStore(fname) as storedata:
        storedata.append(key, df)

with pd.HDFStore(fname) as storedata:
    storedata.get_storer(key).attrs.metadata=metadata

In [9]:
fname = 'lr_stss_hyperparameter_study.hdf5'
key = 'grid'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)
with pd.HDFStore(fname) as storedata:
    df = storedata[key]
# display(HTML(df.to_html()))
df.head()

,graph_type,auc,rec_err,anomaly_cardinality,number_of_anomalies,anomaly_duration,anomaly_radius,graph_seed,anomaly_seed,anomaly_amplitude,...,lda_2,lda_l,lda_t,psi_1,psi_2,psi_3,psi_4,maxit,maxit2,it
0,grid,0.869966,0.233005,1571,100,4,1,46513213,46513213,1,...,100,10.000000,10.000000,20,20,20,20,300,40,300
1,grid,0.761745,0.233037,1571,100,4,1,46513213,46513213,1,...,100,10.000000,0.215443,20,20,20,20,300,40,300
2,grid,0.906628,0.234891,1571,100,4,1,46513213,46513213,1,...,100,10.000000,0.004642,20,20,20,20,300,40,300
3,grid,0.830363,0.235160,1571,100,4,1,46513213,46513213,1,...,100,10.000000,0.000100,20,20,20,20,300,40,300
4,grid,0.828935,0.233010,1571,100,4,1,46513213,46513213,1,...,100,0.215443,10.000000,20,20,20,20,300,40,300


In [10]:
import seaborn as sns
corr_spearman = df.corr(method='spearman')
corr_spearman

C:\Users\merti\AppData\Local\Temp\ipykernel_140496\3745845495.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_spearman = df.corr(method='spearman')


,auc,rec_err,anomaly_cardinality,number_of_anomalies,anomaly_duration,anomaly_radius,graph_seed,anomaly_seed,anomaly_amplitude,lda_1,lda_2,lda_l,lda_t,psi_1,psi_2,psi_3,psi_4,maxit,maxit2,it
auc,1.000000,-0.375706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.377574,NaN,0.351065,-0.244804,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rec_err,-0.375706,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.432829,NaN,-0.494989,-0.469432,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly_cardinality,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
number_of_anomalies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly_duration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly_radius,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
graph_seed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly_seed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly_amplitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lda_1,0.377574,-0.432829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
dfm = df.groupby(['lda_1', 'lda_t', 'lda_l'])[['auc', 'rec_err']].mean()
dfm

auc   rec_err
lda_1     lda_t     lda_l                        
0.000100  0.000100  0.000100   0.680665  1.000103
                    0.004642   0.680663  1.000147
                    0.215443   0.681124  0.999453
                    10.000000  0.764031  0.297027
          0.004642  0.000100   0.680667  1.000158
                    0.004642   0.680667  1.000202
                    0.215443   0.681148  0.999407
                    10.000000  0.767154  0.294271
          0.215443  0.000100   0.681468  0.999407
                    0.004642   0.681464  0.999455
                    0.215443   0.681834  0.999323
                    10.000000  0.927227  0.193813
          10.000000 0.000100   0.632456  0.298692
                    0.004642   0.634085  0.296600
                    0.215443   0.754813  0.224587
                    10.000000  0.605585  0.230240
0.031623  0.000100  0.000100   0.680619  1.000333
                    0.004642   0.680619  1.000375
                    0.215443   0.681162  0.999305
                    10.000000  0.776148  0.285049
          0.004642  0.000100   0.680622  1.000386
                    0.004642   0.680623  1.000427
                    0.215443   0.681177  0.999294
                    10.000000  0.779455  0.282374
          0.215443  0.000100   0.681383  0.999737
                    0.004642   0.681369  0.999817
                    0.215443   0.681638  1.000215
                    10.000000  0.934498  0.188496
          10.000000 0.000100   0.640014  0.287765
                    0.004642   0.641901  0.285774
                    0.215443   0.767726  0.220925
                    10.000000  0.602974  0.230221
10.000000 0.000100  0.000100   0.811154  0.242206
                    0.004642   0.833236  0.241473
                    0.215443   0.902281  0.235996
                    10.000000  0.830363  0.235160
          0.004642  0.000100   0.855289  0.241643
                    0.004642   0.879548  0.240937
                    0.215443   0.949797  0.235664
                    10.000000  0.906628  0.234891
          0.215443  0.000100   0.853711  0.235307
                    0.004642   0.897150  0.234873
                    0.215443   0.626503  0.233214
                    10.000000  0.761745  0.233037
          10.000000 0.000100   0.595792  0.233724
                    0.004642   0.639127  0.233541
                    0.215443   0.828935  0.233010
                    10.000000  0.869966  0.233005

2023-06-26 08:36:35,739 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\merti\miniconda3\lib\site-packages\tornado\websocket.py", line 942, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "c:\Users\merti\miniconda3\lib\site-packages\tornado\web.py", line 3208, in wrapper
    return method(self, *args, **kwargs)
  File "c:\Users\merti\miniconda3\lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


In [22]:
df[['auc','rec_err','lda_1','lda_t','lda_l']].corr('spearman')
# plt.figure(figsize=(7,6))
# sns.heatmap(df[['auc','rec_err','lda_1','lda_t','lda_l']].corr('spearman'))

,auc,rec_err,lda_1,lda_t,lda_l
auc,1.000000,-0.375706,0.377574,-0.244804,0.351065
rec_err,-0.375706,1.000000,-0.432829,-0.469432,-0.494989
lda_1,0.377574,-0.432829,1.000000,0.000000,0.000000
lda_t,-0.244804,-0.469432,0.000000,1.000000,0.000000
lda_l,0.351065,-0.494989,0.000000,0.000000,1.000000


In [9]:
client.shutdown()

2023-06-26 05:43:22,832 - distributed.nanny - WARNING - Worker process still alive after 3.1999905395507815 seconds, killing
2023-06-26 05:43:22,834 - distributed.nanny - WARNING - Worker process still alive after 3.1999929809570316 seconds, killing
2023-06-26 05:43:22,836 - distributed.nanny - WARNING - Worker process still alive after 3.1999951171875 seconds, killing
2023-06-26 05:43:22,888 - distributed.nanny - ERROR - Error in Nanny killing Worker subprocess
Traceback (most recent call last):
  File "c:\Users\merti\miniconda3\lib\asyncio\tasks.py", line 418, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\merti\miniconda3\lib\site-packages\distributed\nanny.py", line 597, in close
    await self.kill(timeout=timeout, reason=reason)
  File "c:\Users\merti\miniconda3\lib\site-packages\distributed\nanny.py", line 380, in kill
    await self.p

2023-06-26 05:43:22,979 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000002406FEC1E40>>, <Task finished name='Task-74123' coro=<MultiProgressBar.listen() done, defined at c:\Users\merti\miniconda3\lib\site-packages\distributed\diagnostics\progressbar.py:269> exception=CommClosedError('in <TCP (closed)  local=tcp://127.0.0.1:55018 remote=tcp://127.0.0.1:54740>: Stream is closed')>)
Traceback (most recent call last):
  File "c:\Users\merti\miniconda3\lib\site-packages\distributed\comm\tcp.py", line 225, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\merti\miniconda3\lib\site-packages\tornado\ioloop.py", line 740, in _run_callback
    ret = callback()
  File "c:\Users\merti\m